<a href="https://colab.research.google.com/github/Samin-Sadaf7/QuestionAnswerGenerator/blob/main/TopicDatabase_and_Further.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install --quiet pandas pymongo sentence_transformers transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 59.9 MB/s eta 0:00:00


In [3]:
!python -m pip install "pymongo[srv]"

In [4]:
import pandas as pd

In [5]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from google.colab import userdata

uri = userdata.get('MongoDB_URL')

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [6]:
db = client['RAG_application']

In [26]:
from pymongo import MongoClient
from pymongo.errors import CollectionInvalid

# Define the collection name
topic_name = 'topic'

# Define the question schema
topic_schema = {
    'prof': {
        'type': 'string',
        'required': True,
    },
    'term':{
        'type': 'string',
    },
    'subject': {
        'type': 'string',
        'required': True,
    },
    'card': {
        'type': 'string',
        'required': True,
    },
    'topic': {
        'type': 'string',
        'required': True,
    },
    'subTopic': {
        'type': 'string',
    }
}

# Initialize the validator
validator = {
    '$jsonSchema': {
        'bsonType': 'object',
        'properties': {}
    }
}

required = []

def process_field(field_key, field, parent_required, parent_properties):
    properties = {'bsonType': field['type']}
    if 'min_length' in field:
        properties['minLength'] = field['min_length']
    if 'required' in field and field['required']:
        parent_required.append(field_key)
    parent_properties[field_key] = properties

for field_key, field in topic_schema.items():
    process_field(field_key, field, required, validator['$jsonSchema']['properties'])

if required:
    validator['$jsonSchema']['required'] = required


# Create the collection with the validator
try:
    db.create_collection(topic_name, validator=validator)
    print(f"Collection '{topic_name}' created with validator.")
except CollectionInvalid:
    print(f"Collection '{topic_name}' already exists or validation failed.")

print(validator)

Collection 'topic' created with validator.
{'$jsonSchema': {'bsonType': 'object', 'properties': {'prof': {'bsonType': 'string'}, 'term': {'bsonType': 'string'}, 'subject': {'bsonType': 'string'}, 'card': {'bsonType': 'string'}, 'topic': {'bsonType': 'string'}, 'subTopic': {'bsonType': 'string'}}, 'required': ['prof', 'subject', 'card', 'topic']}}


In [27]:
topics_csv= pd.read_csv('/content/gdrive/MyDrive/AI_integration_kernel/MediBee_MBBS_AI/all_topics.csv')

In [28]:
topics_df = pd.DataFrame(topics_csv)

In [29]:
topics_df.columns

Index(['prof', 'subject', 'card', 'topic', 'subTopic'], dtype='object')

In [30]:
topic_documents = topics_df.to_dict('records')

In [ ]:
topic_documents

In [32]:
db.list_collection_names()

['student_collection_updated',
 'engi_question_collection',
 'topic',
 'Medical_Books_RAG',
 'question_collection']

In [33]:
results = db.topic.insert_many(topic_documents)
print(f"Inserted {len(results.inserted_ids)} documents into the collection.")

Inserted 485 documents into the collection.
